In [1]:
#https://github.com/crawfordsm/specidentify/blob/master/docs/notebook/arcwidget_tutorial.ipynb

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
# imports needed from 
import numpy as np
from matplotlib import pyplot as plt

from astropy import units as u
from astropy import modeling as mod
from astropy.io import fits
from astropy.table import Table

from specutils import Spectrum1D

In [3]:
# imports needed from specidentify 
from specidentify import WavelengthSolution
from specidentify import ArcIdentify

In [4]:
lamp_image = fits.getdata('MKD_20211104.0068.fits')
lamp_spec = np.flip(np.median(lamp_image[330:340, :], axis=0))

print (np.shape(lamp_spec))

arc = Spectrum1D(spectral_axis=np.arange(len(lamp_spec))*u.pixel , flux=lamp_spec*u.electron)

(1024,)


In [5]:
# a plot of the uncalibrated arc spectra
ax = plt.subplots()[1]  
ax.plot(arc.spectral_axis, arc.flux)  
ax.set_xlabel("Dispersion [pixels]")  
ax.set_ylabel("Flux [electron]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Flux [electron]')

In [6]:
xarr = np.arange(len(arc.data))

line_table = np.loadtxt('Xe.txt', unpack=True, usecols=(0,1))
line_table = Table(line_table.T, names=('wavelength', 'flux'))

In [7]:
ws_init = mod.models.Chebyshev1D(3)
ws_init.domain = [xarr.min(), xarr.max()]
ws_init = mod.fitting.LinearLSQFitter()(ws_init, xarr, xarr)

In [8]:
ws = WavelengthSolution(x=None, wavelength=None, model=ws_init)

In [9]:
x = [134,180,483,574,700,813,930,1008]
w = [4500,4671,5823,6318,6668,7119,7584,7887]
ws = WavelengthSolution(x=x, wavelength=w, model=ws_init)
ws.fit()

In [10]:
aw = ArcIdentify(arc, ws, line_table, wavelength_range=[4000, 8000], flatten_order=9)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
aw.show_commands()


 x - centroid on line    a - Display spectrum
 b - identify features   f - fit solution
 r - redraw spectrum     e - add closest line
 d - delete feature
       


In [11]:
aw.draw_error()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [101]:
# Displays the rms of the fit
aw.ws.sigma(aw.xp, aw.wp)

2.711763924897241

In [102]:
arc_w = Spectrum1D(spectral_axis=aw.ws(np.arange(len(lamp_spec))) * u.angstrom, flux=lamp_spec*u.electron)

In [103]:
ax = plt.subplots()[1]  
ax.plot(arc_w.spectral_axis, arc_w.flux)  
ax.set_xlabel("Dispersion [Angstrom]")  
ax.set_ylabel("Flux [electron]")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Flux [electron]')

In [104]:
f = open('ref_xe_mk_lo.dat','w')
[f.write('%g %g\n' % (arc_w.spectral_axis[i].angstrom, lamp_spec[i])) for i in range(len(arc_w.spectral_axis))]
f.close()

In [143]:
lamp_image = fits.getdata('tarc.fits')
lamp_spec = np.flip(np.sum(lamp_image[10:20, :], axis=0))

arc = Spectrum1D(spectral_axis=np.arange(len(lamp_spec))*u.pixel , flux=lamp_spec*u.electron)

In [154]:
x = [134,180,483,574,700,813,930,1008]
w = [4500,4671,5823,6318,6668,7119,7584,7887]
ws = WavelengthSolution(x=x, wavelength=w, model=ws_init)
ws.fit()

In [155]:
aw = ArcIdentify(arc, ws, line_table, wavelength_range=[4000, 8000], flatten_order=9)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [162]:
aw.draw_error()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [163]:
arc_w = Spectrum1D(spectral_axis=aw.ws(np.arange(len(lamp_spec))) * u.angstrom, flux=lamp_spec*u.electron)

In [165]:
f = open('ref_xe_mk_hi.dat','w')
[f.write('%g %g\n' % (arc_w.spectral_axis[i].angstrom, 0.19*lamp_spec[i])) for i in range(len(arc_w.spectral_axis))]
f.close()